# Cleaning txt file

In [28]:
import cleanData
replacements = [('\n', ''), ('   ', ' '), ('  ', ' '), ('\"', ''), 
                ('\.+', '.'), ('\?', '.'), ('!', '.'), ('\(',''), ('\)',''), ('-', ' '), (',', ' ')]
cleanData.cleanData(replacements)

name: abc.txt
abc
 
 big a, little a, what begins with a? 
 aunt annie's alligator.....a...a...a 
 big b, little b, what begins with b? 
 barber baby bubbles and a bumblebee. 
 big c, little c, what begins with c? 
 camel on the ceiling c...c....c 
 big d, little d, what begins with d? 
 david donald doo dreamed a doen doughnuts and a duck-dog, too. 
 abcde...e...e 
 ear egg elephant e e e 
 big f, little f 
 f.... f....f 
 four fluffy feathers on a fiffer-feffer-feff. 
 abcdeg 
 goat girl googoo goggles 
 g...g....g 
 big h, little h 
 hungry horse. hay. hen in a hat. hooray! hooray! 
 big i, little i, i.....i...i 
 icabod is itchy. so am i. 
 big j, little j, what begins with j? 
 jerry jordans jelly jar and jam begins that way. 
 big k, little k 
 kitten. kangaroo. kick a ketlle. kite and a king's kerchoo. 
 big l, little l 
 little lola lopp. left leg. lazy lion licks a lollipop. 
 big m, little m 
 many mumbling mice are making midnight music in the moonlight....mighty nice 
 big 

In [23]:
import numpy
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils
import sys
import os

# Training Model

In [29]:
filename = "data/combinedText.txt"
raw_text = open(filename).read()
raw_text

"abc big a  little a  what begins with a. aunt annie's alligator.a.a.a big b  little b  what begins with b. barber baby bubbles and a bumblebee. big c  little c  what begins with c. camel on the ceiling c.c.c big d  little d  what begins with d. david donald doo dreamed a doen doughnuts and a duck dog  too. abcde.e.e ear egg elephant e e e big f  little f f. f.f four fluffy feathers on a fiffer feffer feff. abcdeg goat girl googoo goggles g.g.g big h  little h hungry horse. hay. hen in a hat. hooray. hooray. big i  little i  i.i.i icabod is itchy. so am i. big j  little j  what begins with j. jerry jordans jelly jar and jam begins that way. big k  little k kitten. kangaroo. kick a ketlle. kite and a king's kerchoo. big l  little l little lola lopp. left leg. lazy lion licks a lollipop. big m  little m many mumbling mice are making midnight music in the moonlight.mighty nice big n  little n  what begins with n. nine new neckties and a nigthshirt and a nose. o is very useful. you use it 

In [32]:
list(set(raw_text))
print(len(list(set(raw_text))))

44


In [33]:
# create mapping of unique chars to integers
chars = sorted(list(set(raw_text)))
char_to_int = dict((c, i) for i, c in enumerate(chars))
char_to_int

{' ': 0,
 "'": 1,
 '.': 2,
 '/': 3,
 '1': 4,
 '2': 5,
 '3': 6,
 '4': 7,
 '5': 8,
 '6': 9,
 '8': 10,
 '9': 11,
 ':': 12,
 ';': 13,
 'a': 14,
 'b': 15,
 'c': 16,
 'd': 17,
 'e': 18,
 'f': 19,
 'g': 20,
 'h': 21,
 'i': 22,
 'j': 23,
 'k': 24,
 'l': 25,
 'm': 26,
 'n': 27,
 'o': 28,
 'p': 29,
 'q': 30,
 'r': 31,
 's': 32,
 't': 33,
 'u': 34,
 'v': 35,
 'w': 36,
 'x': 37,
 'y': 38,
 'z': 39,
 '’': 40,
 '“': 41,
 '”': 42,
 '…': 43}

In [34]:
n_chars = len(raw_text)
n_vocab = len(chars)
print ("Total Characters: ", n_chars)
print ("Total Vocab: ", n_vocab)

Total Characters:  55673
Total Vocab:  44


In [44]:
# prepare the dataset of input to output pairs encoded as integers
seq_length = 100
dataX = []
dataY = []
for i in range(0, n_chars - seq_length, 1):
    seq_in = raw_text[i: i + seq_length]
    seq_out = raw_text[i + seq_length]
    dataX.append([char_to_int[char] for char in seq_in])
    dataY.append(char_to_int[seq_out])
    #for visualization
#     if i%5000==0:
#         print(f'i:{i}\nseq_in: {seq_in}\nseq_out: {seq_out}\ndataX: {dataX}\ndataY: {dataY}')
n_patterns = len(dataX)
print ("Total Patterns: ", n_patterns)

i:0
seq_in: abc big a  little a  what begins with a. aunt annie's alligator.a.a.a big b  little b  what begins w
seq_out: i
dataX: [[14, 15, 16, 0, 15, 22, 20, 0, 14, 0, 0, 25, 22, 33, 33, 25, 18, 0, 14, 0, 0, 36, 21, 14, 33, 0, 15, 18, 20, 22, 27, 32, 0, 36, 22, 33, 21, 0, 14, 2, 0, 14, 34, 27, 33, 0, 14, 27, 27, 22, 18, 1, 32, 0, 14, 25, 25, 22, 20, 14, 33, 28, 31, 2, 14, 2, 14, 2, 14, 0, 15, 22, 20, 0, 15, 0, 0, 25, 22, 33, 33, 25, 18, 0, 15, 0, 0, 36, 21, 14, 33, 0, 15, 18, 20, 22, 27, 32, 0, 36]]
dataY: [22]
i:5000
seq_in: f your stars so you won't look like sneetches who have them on thars. and that handy machine working
seq_out:  
dataX: [[14, 15, 16, 0, 15, 22, 20, 0, 14, 0, 0, 25, 22, 33, 33, 25, 18, 0, 14, 0, 0, 36, 21, 14, 33, 0, 15, 18, 20, 22, 27, 32, 0, 36, 22, 33, 21, 0, 14, 2, 0, 14, 34, 27, 33, 0, 14, 27, 27, 22, 18, 1, 32, 0, 14, 25, 25, 22, 20, 14, 33, 28, 31, 2, 14, 2, 14, 2, 14, 0, 15, 22, 20, 0, 15, 0, 0, 25, 22, 33, 33, 25, 18, 0, 15, 0, 0, 36, 21, 14, 33, 0, 15,

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [10]:
# reshape X to be [samples, time steps, features]
X = numpy.reshape(dataX, (n_patterns, seq_length, 1))
# normalize
X = X / float(n_vocab)
# one hot encode the output variable
y = np_utils.to_categorical(dataY)

In [11]:
# define the LSTM model
model = Sequential()
model.add(LSTM(256, input_shape=(X.shape[1], X.shape[2])))
model.add(Dropout(0.2))
model.add(LSTM(128, input_shape=(X.shape[1], X.shape[2])))
model.add(Dropout(0.2))
model.add(LSTM(64, input_shape=(X.shape[1], X.shape[2])))
model.add(Dropout(0.2))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [12]:
# define the checkpoint
filepath="weights-improvement-{epoch:02d}-{loss:.4f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]

In [13]:
model.fit(X, y, epochs=20, batch_size=128, callbacks=callbacks_list, verbose=1)

Epoch 1/20
29947/29947 [==============================] - 2552s 85ms/step - loss: 2.9403

Epoch 00001: loss improved from inf to 2.94030, saving model to weights-improvement-01-2.9403.hdf5
Epoch 2/20
29947/29947 [==============================] - 2829s 94ms/step - loss: 2.8767

Epoch 00002: loss improved from 2.94030 to 2.87673, saving model to weights-improvement-02-2.8767.hdf5
Epoch 3/20
29947/29947 [==============================] - 2434s 81ms/step - loss: 2.7569

Epoch 00003: loss improved from 2.87673 to 2.75686, saving model to weights-improvement-03-2.7569.hdf5
Epoch 4/20
29947/29947 [==============================] - 2367s 79ms/step - loss: 2.6688

Epoch 00004: loss improved from 2.75686 to 2.66882, saving model to weights-improvement-04-2.6688.hdf5
Epoch 5/20
29947/29947 [==============================] - 2677s 89ms/step - loss: 2.6138

Epoch 00005: loss improved from 2.66882 to 2.61383, saving model to weights-improvement-05-2.6138.hdf5
Epoch 6/20
29947/29947 [===============

# Running Model with Seed Input

In [14]:
filename = "weights-improvement-20-1.7518.hdf5"
model.load_weights(filename)
model.compile(loss='categorical_crossentropy', optimizer='adam')



int_to_char = dict((i, c) for i, c in enumerate(chars))

pattern = "the cat in the hat said to the bat that i’m fat. oh no, said the bat, i’m not fat. what say you bird"

x_train = [char_to_int[i] for i in pattern]
x_train = numpy.reshape(x_train, (1,len(x_train),1))
x_train = x_train / float(n_vocab)

for i in range(50):
    x_train = numpy.reshape(x_train, (1,-1,1))
    prediction = model.predict(x_train, verbose=0)
    index = numpy.argmax(prediction)
    result = int_to_char[index]
#     seq_in = [int_to_char[value] for value in x_train]

    sys.stdout.write(result)
    x_train = numpy.append(x_train, index)
    x_train = x_train[1:len(x_train)]
    
    

 th lh lh lh lh lh lh lh lh lh lh lh lh lh lh lh l

In [ ]:
temp = model.predict(numpy.reshape(x_train, (1,-1,1)))

In [ ]:
print(list(zip(temp[0], int_to_char.values())))

In [ ]:
int_to_char[15]
int_to_char[26]